This notebook goes through how to draw a ***braid*** by **elongating** and **twisting** a **torus**.

# IMPORTS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go

from spomso.cores.helper_functions import generate_grid, smarter_reshape, hard_binarization
from spomso.cores.geom import GenericGeometry
from spomso.cores.geom_3d import Torus

# PARAMETERS

In [ ]:
# size of the volume
co_size = 1.5, 1.5, 4
# resolution of the volume
co_resolution = 150, 150, 200

show = "BINARY" # BINARY, FIELD
show_midplane = True
show_3d = True

# COORDINATE SYSTEM

Point cloud of coordinates is created with the **generate_grid** function, which takes the size of coordinate system and the number of points in each direction (**co_resolution**).

In [ ]:
coor, co_res_new = generate_grid(co_size, co_resolution)

If there is an even number of points along any direction the function will automatically convert it to an odd number, therefore the origin is always included in the point cloud. The point cloud is a numpy.ndarray with a shape of **(D, N)**, where **D** is the number of dimensions and **N** is the number of data points.

# CREATE SDFs

Create the **torus** with a the major radius of **0.25** and the minor radius of **0.2**:

In [ ]:
torus = Torus(0.25, 0.2)

Elongate the **torus** along the **x-axis** for 4 units to create a **chin link**:

In [ ]:
torus.elongation((4., 0., 0.0))

**Rotate** the **torus** by **90°** around the **y-axis**

In [ ]:
torus.rotate(np.pi/2, (0, 1, 0))

Print the **modifications** and **transformations** applied to the **torus**:

In [ ]:
print("Torus modifications:", torus.modifications)
print("Torus transformations:", torus.transformations)

Extract the SDF of the **modified torus** and create **new geometry** from the extracted SDF:

In [ ]:
modified_torus_sdf = torus.propagate
braid = GenericGeometry(modified_torus_sdf, ())

Twist the new geometry **(chain link)** with a **pitch** of **π/(unit length)** to create a **braid**:

In [ ]:
braid.twist(np.pi)

Print the **modifications** and **transformations** applied to the **chain link** object:

In [ ]:
print("Braid modifications:", braid.modifications)
print("Braid transformations:", braid.transformations)

## EVALUATE

Evaluate the SDF of the ***braid*** to create a signed distance field 3D map. This is done by calling the **create** method and passing in the point cloud of coordinates:

In [ ]:
braid_pattern = braid.create(coor)

# BINARIZATION

Convert the distance field to a binary voxel map, where 1 corresponds to the interior and 0 to the exterior of the geometry.

In [ ]:
if show_midplane:
    field = smarter_reshape(braid_pattern, co_resolution)
    if show=="BINARY":
        pattern_2d = hard_binarization(field, 0)

if show=="BINARY":
    pattern = hard_binarization(braid_pattern, 0)

# PLOT

In [ ]:
print("Drawing results...")
# Mid-plane cross-section plot
if show_midplane and show=="BINARY":
    fig, ax = plt.subplots(1,1, figsize=(8.25, 8.25))
    ax.imshow(pattern_2d[:, :, co_resolution[-1]//2].T,
              cmap="binary_r",
              extent=(-co_size[0]/2, co_size[0]/2,
                      -co_size[1]/2, co_size[1]/2),
              origin="lower"
              )
    ax.grid()

    fig.tight_layout()
    plt.show()

if show_midplane and show == "FIELD":
    fig, ax = plt.subplots(1, 1, figsize=(8.25, 8.25))
    ax.imshow(field[:, :, co_resolution[-1] // 2].T,
              cmap="binary_r",
              extent=(-co_size[0] / 2, co_size[0] / 2,
                      -co_size[1] / 2, co_size[1] / 2),
              origin="lower"
              )
    z_mask = coor[2] == 0
    cs = ax.contour(coor[0, z_mask].reshape(co_res_new[0], co_res_new[1]),
                    coor[1, z_mask].reshape(co_res_new[0], co_res_new[1]),
                    field[:, :, co_resolution[-1] // 2],
                    cmap="plasma_r")
    ax.clabel(cs, inline=True, fontsize=10)
    ax.grid()

    fig.tight_layout()
    plt.show()

In [ ]:
# Isosurfaces plot
if show_3d and show=="BINARY":
    fig = go.Figure(data=go.Volume(
        x=coor[0],
        y=coor[1],
        z=coor[2],
        value=pattern,
        isomin=0.1,
        isomax=1,
        opacity=0.1,
        surface_count=2,
    ))
    fig.show()

if show_3d and show=="FIELD":
    fig = go.Figure(data=go.Volume(
        x=coor[0],
        y=coor[1],
        z=coor[2],
        value=braid_pattern,
        isomin=-0,
        isomax=0.5,
        opacity=0.1,
        surface_count=5,
    ))
    fig.show()

